In [ ]:
#r "nuget: FParsec, 1.1.1"

open FParsec
open System.IO
open System.Text



Installed Packages FParsec, 1.1.1

In [ ]:
let pInt = pint32 |>> int
let pTuple = (pInt .>> pstring ",") .>>. pInt
let pLine = (pTuple .>> pstring " -> ") .>>. pTuple
let pAll = sepBy pLine newline

let data =
    match runParserOnFile pAll () "day5.txt" Encoding.ASCII with
    | Success (result, _, _) -> result
    | _ -> failwith "Couldn't parse input"

let getPoints = function
| ((x1, y1), (x2, y2)) when y1 < y2 -> seq { for i in y1..y2 do (x1, i) }
| ((x1, y1), (x2, y2)) when y1 > y2 -> seq { for i in y2..y1 do (x1, i) }
| ((x1, y1), (x2, y2)) when x1 < x2 -> seq { for i in x1..x2 do (i, y1) }
| ((x1, y1), (x2, y2)) -> seq { for i in x2..x1 do (i, y1) }

let day5part1solution =
    data
    |> List.filter (fun (p1, p2) -> fst p1 = fst p2 || snd p1 = snd p2)
    |> List.collect (getPoints >> Seq.toList)
    |> List.countBy id
    |> List.filter (fun p -> snd p >= 2)
    |> List.length

day5part1solution

5145